In [1]:
# General Libraries
import numpy as np
import pandas as pd

# Web Scraping Libraries
import urllib
import requests
from bs4 import BeautifulSoup

# Regex Library
import re

# Time-related library
import time

### URL Format of Indeed.com
1. Search chemist in TX<br>
https://www.indeed.com/jobs?q=chemist&l=TX
2. Search chemist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=chemist&l=San+Antonio%2C+TX
3. Search data scientist in San Antonio, TX<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX
4. Search data scientist intern in San Anotnio, TX<br>
https://www.indeed.com/jobs?q=data+scientist+intern&l=San+Antonio%2C+TX
5. Sort the data scientist jobs posting by date<br>
https://www.indeed.com/jobs?q=data+scientist&l=San+Antonio%2C+TX&sort=date

**Takeaways**
1. q = job title
2. l = location

### URL Format of Monster.com
https://www.monster.com/jobs/search/?q=data-scientist&where=San-Antonio__2C-TX

### Generate the URL of a Job Search at Indeed.com

In [2]:
def first_page_url_indeed(job_title, location):
    '''
    This function returns a URL of the 1st page of a job search at Indeed.com 
    based on the job title and the location.
    '''
    # Create the base URL for a job serch at Indeed.com
    base_url = 'https://www.indeed.com/jobs?'
    # Create a dictionary to map the keys to the input parameters
    dic = {'q': job_title, 'l': location, 'sort': 'date'}
    # Convert the dictionary to a query string
    relative_url = urllib.parse.urlencode(dic)
    # Generate the full URL of the first page
    url = base_url + relative_url
    return url

In [42]:
# Test the function
url = first_page_url_indeed('data scientist', 'al')
url

'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'

In [97]:
def urls_indeed(job_title, location):
    '''
    This function returns all the URLs in a job searching result.
    '''
    # Create a variable urls to hold the URLs of all pages
    urls = []
    # Generate the URL of the first page
    first_page_url = first_page_url_indeed(job_title, location)
    # Append the URL of the first page
    urls.append(first_page_url)
    # Generate the Soup object of the first page
    first_page_soup = first_page_soup_indeed(job_title, location)
    # Compute the total number of jobs based on the search
    num_jobs = num_jobs_indeed(first_page_soup) 
    # Estimate the total number of pages based on 15 job cards each page
    num_page = round(int(num_jobs)/15)
    # For Loop through all the pages to generate their URLs
    for i in range(1, num_page):
        dic = {'start': i*10}
        relative_url = urllib.parse.urlencode(dic)
        url = first_page_url + '&' + relative_url
        urls.append(url)
    return urls

In [120]:
urls_indeed('data scientist', 'md')

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Maryland | Indeed.com


['https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=10',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=20',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=30',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=40',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=50',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=60',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=70',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=80',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=90',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=100',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=110',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=date&start=120',
 'https://www.indeed.com/jobs?q=data+scientist&l=md&sort=

### Make the HTTP Request

In [49]:
def first_page_soup_indeed(job_title, location):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of the first page of a request for job searching at Indeed.com
    '''
    # Generate the URL of the job search based on title and location
    url = first_page_url_indeed(job_title, location)
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [52]:
first_page_soup = first_page_soup_indeed("data scientist", 'al')
type(first_page_soup)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com


bs4.BeautifulSoup

In [12]:
# Find out the tag that contains the number of the jobs by seaching

num_jobs = first_page_soup.find('div', id='searchCountPages')
print("Data Type: ", type(num_jobs))
print("Name of the Tag: ", num_jobs.name)
print("Attributes of the Tag: ", num_jobs.attrs)
print("Text within the Tag: ")
num_jobs.text

Data Type:  <class 'bs4.element.Tag'>
Name of the Tag:  div
Attributes of the Tag:  {'id': 'searchCountPages'}
Text within the Tag: 


'\n                    Page 1 of 560 jobs'

In [13]:
# Find the number of the jobs in the text
match = re.findall(r'(\d+)', num_jobs.text)
match[1]

'560'

In [9]:
def num_jobs_indeed(first_page_soup):
    '''
    This function returns the total number of the jobs in the searching result.
    '''
    # Find out the section contains total number of jobs  
    div = first_page_soup.find('div', id='searchCountPages')
    # Extract the number
    num_jobs = re.findall(r'(\d+)', div.text)[1]
    return num_jobs

In [53]:
# Test the function num_jobs_indeed
num_jobs_indeed(first_page_soup)

'40'

In [12]:
def page_num_indeed(soup):
    '''
    This function returns the page number of job searching results. 
    '''
    # Find out the section contains total number of jobs  
    div = soup.find('div', id='searchCountPages')
    # Extract the number
    page_num = re.findall(r'(\d+)', div.text)[0]
    return page_num

In [54]:
# Test the function num_jobs_indeed
page_num_indeed(first_page_soup)

'1'

In [14]:
# Define a function to extract all job cards in a Indeed page

def job_cards_indeed(soup):
    '''
    This function accepts the Soup object of a Indeed page 
    return an iterator containing the all the job cards in this page.
    '''
    # Find the appropriate tag that contains all of the job listings in this page
    tag = soup.find('td', id="resultsCol")
    # Extract all job cards
    job_cards = tag.find_all('div', class_='jobsearch-SerpJobCard')
    return job_cards

In [55]:
job_cards = job_cards_indeed(first_page_soup)
# Print the data type of job_cards
type(job_cards)

bs4.element.ResultSet

**Quick Note**: job_cards is an iterator

In [56]:
# How many jobs listed in the 1st page? 
len(job_cards)

15

In [17]:
def job_titles_indeed(job_cards):
    '''
    This function extract the job titles from a job_cards set. 
    '''
    # Create a list to hold the job titles
    titles = []
    # For Loop throught the job cards to extract the titles
    for job in job_cards:
        title = job.find('h2', class_='title')
        title = title.text.strip()
        titles.append(title)
    return titles

In [57]:
titles = job_titles_indeed(job_cards)
titles

['Data Scientist\nnew',
 'MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE\nnew',
 'Data Scientist\nnew',
 'Data Analyst - Microsoft Stack (mid-senior)\nnew',
 'Statistical Analyst\nnew',
 'Data Scientist Intern\nnew',
 'Machine Learning/Artificial Intelligence Software Developer\nnew',
 'Software Engineer/Data Scientist\nnew',
 'Lead Financial Analyst - Artificial Intelligence Strategic G...\nnew',
 'BI Architect/Data Scientist',
 'Asst Research Professional - Research Data Scientist',
 '2021-18 Software Engineers for BMDS Data Analysis Suite',
 'Cyber Artificial Intelligence (AI) SME',
 '2021-02 Artificial Intelligence Designer',
 'Deep Learning Engineer']

In [19]:
# Define a function to pull the company names from a set of job cards

def company_names_indeed(job_cards):
    '''
    This function extracts the company names from a set of job cards.
    '''
    # Create a list to hold the company names
    names = []
    # For loop through the job cards to pull the company names
    for job in job_cards:
        name = job.find('span', class_='company')
        name = name.text.strip()
        names.append(name)
    return names

In [58]:
# Test the function: comany_names_indeed

company_names = company_names_indeed(job_cards)
company_names

['Quiq Inc',
 'B.A.S.S., LLC',
 'Vision',
 'Vaco',
 'The Personnel Board of Jefferson County',
 'LOCKHEED MARTIN CORPORATION',
 'IERUS Technologies, Inc.',
 'Torch Technologies, Inc.',
 'Deloitte',
 'Doozer Software',
 'The University of Alabama',
 '1st Edge',
 'Quantum Research International, Inc.',
 '1st Edge',
 'Numerator']

In [21]:
# Define a function to pull the post ages from a set of job cards

def post_ages_indeed(job_cards):
    '''
    This function pulls the post ages from a set of job cards.
    '''
    # Create a list to hold the post ages
    ages = []
    # For loop through the job cards to pull the post ages
    for job in job_cards:
        age = job.find('span', class_='date')
        age = age.text.strip()
        ages.append(age)
    return ages

In [59]:
# Test the function: post_ages_indeed
ages = post_ages_indeed(job_cards)
ages

['Today',
 'Today',
 'Today',
 'Today',
 '2 days ago',
 '4 days ago',
 '5 days ago',
 '6 days ago',
 '6 days ago',
 '11 days ago',
 '11 days ago',
 '12 days ago',
 '22 days ago',
 '21 days ago',
 '28 days ago']

In [23]:
# Define a function to pull the location from a set of job cards

def job_locations_indeed(job_cards):
    '''
    This function pulls the job locations from a set of job cards.
    '''
    # Create a list to hold the locations
    locations = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        location = job.find('div', class_='location accessible-contrast-color-location')
        if location == None:
            location = job.find('span', class_='location accessible-contrast-color-location')
        location = location.text.strip()
        locations.append(location)
    return locations

In [60]:
locations = job_locations_indeed(job_cards)
locations

['United States',
 'Birmingham, AL 35243',
 'Huntsville, AL',
 'Hartselle, AL',
 'Jefferson County, AL',
 'Huntsville, AL 35806',
 'Huntsville, AL 35805',
 'Huntsville, AL 35802',
 'Birmingham, AL 35203 (Central City area)',
 'Birmingham, AL 35216',
 'Tuscaloosa, AL',
 'Huntsville, AL',
 'Huntsville, AL 35806',
 'Huntsville, AL',
 'Alabama']

In [25]:
# Define a function to pull the company ratings from a set of job cards

def company_rating_indeed(job_cards):
    '''
    This function pulls the company rating from a set of job cards.
    If the rating is unavailable, it will be marked as 'missing'.
    '''
    # Create a list to hold the locations
    ratings = []
    # For loop through the job cards to pull the locations
    for job in job_cards:
        rating = job.find('span', class_='ratingsContent')
        if rating == None:
            ratings.append('missing')
            continue
        rating = rating.text.strip()
        ratings.append(rating)
    return ratings

In [61]:
ratings = company_rating_indeed(job_cards)
ratings

['missing',
 'missing',
 'missing',
 '3.7',
 'missing',
 '4.0',
 '4.7',
 'missing',
 '4.0',
 '4.8',
 '4.4',
 'missing',
 '4.0',
 'missing',
 '3.6']

In [27]:
def acuqire_indeed_job_description(url):
    '''
    This function accepts the URL of a job posting and pull its description.
    '''
    # Make the HTTP request
    request = requests.get(url)
    print("Status Code: ", request.status_code)
    # Take a break
    time.sleep(5)
    # Make a soup variable holding the response content
    soup = BeautifulSoup(request.content, "html.parser")
    if soup == None:
        description = 'error'
    else:
        # Print the page's title
        print(soup.title.string)
        # Find the section that contains job description
        description = soup.find('div', id="jobDescriptionText")
        if description == None:
            description = 'error'
        else:
            description = description.text
    return description

def job_links_and_contents_indeed(job_cards):
    '''
    This function pulls the job links and descriptions from a set of job cards.
    '''
    # Create a list to hold the links and descriptions
    links = []
    descriptions = []
    # For loop through the job cards to pull the links and descriptions
    for job in job_cards:
        link = job.find('a')['href']
        link = 'https://www.indeed.com' + link
        link = link.replace(';', '&')
        description = acuqire_indeed_job_description(link)
        links.append(link)
        descriptions.append(description)
    return links, descriptions

In [62]:
# Test the function: job_links_and_contents_indeed
links, descriptions = job_links_and_contents_indeed(job_cards)

Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Code:  200
Lead Financial Analyst - Artificial Intelligence Strategic Growth Offering (AI SGO) Finance & Investment - Birmingham, AL 35203 - Indeed.com
Status Code:  200
BI Architect/Data Scientist - Birmingham, AL 35216 - Indeed.com
Status Code:  200
Asst Research Professional - Research Data Scientist - Tuscaloosa, AL - Indeed.com
Status Code:  200
2021-18 S

In [63]:
links

['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B9JzWmK0GDxRzYzuZf9xSyXN8pQP8ihv6GH-rkAji3LyaR-hLXYB_NfHRnutLWTuqKBdImdeUix5pYB3Uv4mWnHLZlzd5py00o1lMFznkqSTkX2n8aY8mwnVhJALIJXce_3bj6Yi5-UDH-VhWVn3o6zeb2Pm8VmBf7XYjX1VtegcSkVtTpQAVHk_cSXYlbbb9ppduWw_Qf5XW46x30WeNoo3_22bLgKZLzhJEV_E3Ixv1DiqZPYzqp0IMYZ0Djwgv954h1Bqmz53jK2UmTAk6lmmvbIoTkfhAW5z9S56qUXHWlwvic9MvReVXyrEpYz_zaY4XbmDDXr3uIHFCGuQKnKzNsLz5vp9EkzV1o5kbpCj6qti0RZ7NseG1iXFMZ2GODxurbs811H733Ly8Iz-Y8CM0wqrwTs_8vPgE0g-b_Tx-QYca5xn1D&p=0&fvj=1&vjs=3',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0DVr7NBsl4cHSE4aBurKzaRDI_6xQvAm7MDK25NP7GKwrPFBHjnIhnnMkAKtOTcAVcwvdJYQ1iX_OUlRkCcJ9lYHn6yw2Xoq1V-R4Wdf5ZtG2pRL3zKYjPaUIJ7FfKiCmO7nZu9zD3akcI0SzfFz7oRw23qh6o47s6hI0zsqL0wfiKJ4EFcwcqvgr2zn3ygOEqvyjn_BqruYjouMp_OPBKXN1sJf_xD6jAidmRKsWVQCbimyT8pNek2EIcd1SHGimJOFbqnE5QnWqRtOwd1Em2mVe_WF7eRHtG2Ad9d5Btcaz-YQLMoJ5pB8XdgfSqvtHMJyTpI5ilFsZgBFjZL1WHSJK1XUvqlXQd2hYnYwqyUbescsJP9n5A1Qu-e6G6MmRKAdUa65Lq5aeDjyXXxYGCtPmDVqGApWN3X_1kyn3niS0FsFkisS

In [64]:
descriptions[1]

"MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCEBasic Duties & Responsibilities: Query and report on dataBuild and maintain analytical tools, metrics & models across the following channels: membership, sales, marketing and tournaments.Create compelling data visualizations and stories across internal LOB’sResearch internal data in order to build out analytics for both sales and marketing benefitsOwnership of data flow management between our internal systems and 3rd party systems such as PCD, Dovetail, Salesforce, US Sweeps and morePresent information to organizational stakeholders and executivesQualifications: Bachelor’s Degree in mathematics, engineering, computer science, data science, or other relevant fieldStrong written & verbal communication as well as visualization skillsProficiency in analytical/statistical methods and technologies (e.g. SQL, R, Python, Tableau, Microsoft BI, etc)Familiarity with concepts such as predictive modelling, cloud computing, machine learning, artificial inte

In [35]:
# Define a function to create a Soup object based on a job search url

def page_soup_indeed(url):
    '''
    This function returns a BeautifulSoup object to hold the content 
    of a page for a job searching results at Indeed.com
    '''
    # Make the HTTP request
    response = requests.get(url)
    # Print the status code of the request
    print("Status code of the request: ", response.status_code)
    # Sanity check to make sure the document type is HTML
    print("Document type: ", response.text[:15])
    # Take a break
    time.sleep(5)
    # Make a soup to hold the response content
    soup = BeautifulSoup(response.content, "html.parser")
    # Print out the title of the content
    print("Title of the response: ", soup.title.string)
    return soup

In [84]:
# Test the function: page_soup_indeed

url = 'https://www.indeed.com/jobs?q=data+scientist&l=al&sort=date'
soup = page_soup_indeed(url)
type(soup)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com


bs4.BeautifulSoup

In [86]:
# Find out the page number
int(page_num_indeed(soup))

1

In [38]:
# Pull the job cards from the soup
type(job_cards_indeed(soup))

bs4.element.ResultSet

In [99]:
# Define a function to pull job information from a job search URL

def acquire_page_indeed(url):
    '''
    This function accepts a job search URL and returns the page number and
    a pandas dataframe containing job title, location, company, company rating, 
    post age and description. 
    '''
    # Create a Soup object based on the url
    soup = page_soup_indeed(url)
    # Pull the page number
    page_num = page_num_indeed(soup)
    # Pull the job cards
    job_cards = job_cards_indeed(soup)
    # Pull the job titles
    titles = job_titles_indeed(job_cards)   
    # Pull the names of the companies
    companies = company_names_indeed(job_cards)
    # Pull the post ages
    ages = post_ages_indeed(job_cards)
    # Pull the job locations
    locations = job_locations_indeed(job_cards)
    # Pull the company ratings
    ratings = company_rating_indeed(job_cards)
    # Pull the hyperlinks and job description
    links, descriptions = job_links_and_contents_indeed(job_cards)    
    # Create a dataframe
    d = {'title': titles,
         'locations': locations,
         'company': companies, 
         'company_rating': ratings,
         'post_age': ages, 
         'job_link': links, 
         'job_description': descriptions}
    df = pd.DataFrame(d)
    return page_num, df

In [72]:
# Test function acquire_page_indeed
page_num, df = acquire_page_indeed(url)

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Code:  200
Lead Financial Analyst - Artificial Intelligence Strategic Growth Offering (AI SGO) Finance & Investment - Birmingham, AL 35203 - Indeed.

In [73]:
# Print the page number
page_num

'1'

In [79]:
df.columns

Index(['title', 'locations', 'company', 'company_rating', 'post_age',
       'job_link', 'job_description'],
      dtype='object')

In [76]:
df.job_description[0]

"MTA, Inc. is a Woman Owned Small Business with headquarters in Huntsville, AL. We are a diversified company recognized for excellence in Engineering, Integrated Logistics, and Quality Assurance. MTA provides services to U.S. defense agencies, NASA, and the U.S. Corp of Engineers.\nMTA, Inc. has an immediate opening for the position of Data Scientist.\nJob Description:\nTo provide support to the System Readiness Directorate's Reliability, Availability, Maintainability and System Assessment (RAM-SA) Division in support of developing and implementing machine learning algorithms and tools in SQL and Python. The objective of this project is to develop, implement and support data science work in the RAM division in support of tool development research and the Predictive Maintenance National Mission Initiative (PMx NMI).\nDuties and responsibilities:\nPerforming detailed and complex calculations necessary to assess advanced systems concepts\nTransferring data into a new format to make it mor

In [100]:
def jobs_indeed(job_title, location):
    '''
    '''
    # Generate the urls based on job title and location (state)
    urls = urls_indeed(job_title, location)
    # Set up an counter
    counter = 0
    # Create an empty dataframe to hold the job information
    df_jobs = pd.DataFrame(columns = ['title', 'locations', 'company', 'company_rating', 
                                      'post_age','job_link', 'job_description'])
    # For loop through the urls to pull job information
    for url in urls:
        counter = counter+1
        page_num, df = acquire_page_indeed(url)
        print("--------------------------------")
        print("Page: ", counter)
        print("--------------------------------")
        time.sleep(180)
        if int(page_num) == counter:
            df_jobs = df_jobs.append(df, ignore_index=True)
            continue
        if int(page_num) < counter:
            break
    # Print the total number of jobs
    print(f"Total number of {job_title} positions in {location}: ", df_jobs.shape[0])
    return df_jobs

In [101]:
# Test function jobs_in_state_indeed
df = jobs_indeed('data scientist', 'al')

Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status code of the request:  200
Document type:  <!DOCTYPE html>
Title of the response:  Data Scientist Jobs, Employment in Alabama | Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Data Scientist - United States - Indeed.com
Status Code:  200
Vaco Careers and Employment | Indeed.com
Status Code:  200
MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE - Birmingham, AL 35243 - Indeed.com
Status Code:  200
Data Scientist - Huntsville, AL - Indeed.com
Status Code:  200
Statistical Analyst - Jefferson County, AL - Indeed.com
Status Code:  200
Data Scientist Intern - Huntsville, AL 35806 - Indeed.com
Status Code:  200
Machine Learning/Artificial Intelligence Software Developer - Huntsville, AL 35805 - Indeed.com
Status Code:  200
Software Engineer/Data Scientist - Huntsville, AL 35802 - Indeed.com
Status Cod

In [114]:
# Print a concise summary of the df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            42 non-null     object
 1   locations        42 non-null     object
 2   company          42 non-null     object
 3   company_rating   42 non-null     object
 4   post_age         42 non-null     object
 5   job_link         42 non-null     object
 6   job_description  42 non-null     object
dtypes: object(7)
memory usage: 2.4+ KB


In [115]:
df.sample(5)

,title,locations,company,company_rating,post_age,job_link,job_description
17,Member of Technical Staff- Technical Support -...,"Huntsville, AL",Wind River,3.7,30+ days ago,https://www.indeed.com/rc/clk?jk=c768446d7ff0b...,Member of Technical Staff - Technical Support ...
16,Machine Learning Engineer,"Huntsville, AL 35806",CFD Research Corporation,4.2,28 days ago,https://www.indeed.com/rc/clk?jk=189d948471f4a...,This position will join an R&D team focused on...
3,"MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCE\nnew","Birmingham, AL 35243","B.A.S.S., LLC",missing,Today,"https://www.indeed.com/company/B.A.S.S.,-LLC/j...","MANAGER, ACCOUNT DEVELOPMENT/DATA SCIENCEBasic..."
30,AI/ML Software Engineer,"Huntsville, AL",COLSA,3.9,30+ days ago,https://www.indeed.com/rc/clk?jk=fad792fe1641a...,"General Summary\n\n\nDesigns, develops, troubl..."
41,DATA SCIENCE CONSULTANT BIRMINGHAM,"Birmingham, AL 35203 (Central City area)",managementsolutions,3.8,30+ days ago,https://www.indeed.com/rc/clk?jk=aa00dcc168e47...,United States\n\nDATA SCIENCE CONSULTANT BIRMI...


In [112]:
df.job_link[41]

'https://www.indeed.com/rc/clk?jk=aa00dcc168e475b5&fccid=f3eca80b6759548b&vjs=3'

In [113]:
df.job_description[41]

'United States\n\nDATA SCIENCE CONSULTANT BIRMINGHAM\n\nBirmingham / Internship / Number of vacancies: 2\n\n\n\n\nYou will be working in key projects for leading organizations in data mining & knowledge Discovery, predictive modeling, trend modeling, Simulation models (Monte Carlo), Review of credit rating and scoring models and quant support to the business and R&D projects.\n\nRequirements\n\nRecent graduates or final year students from disciplines relating to Mathematics, Physics, Statistics, Econometrics or other Quantitative fields.\nPostgraduate studies and/or specialised courses are an asset, especially in Data Science, Quantitative Finance or similar.\nShould desirably have knowledge of modeling techniques (logit, GLM, time series, decision trees, random forests, clustering), statistical programming languages (SAS, R, Python, Matlab) and big data tools and platforms (Hadoop, Hive, etc.).\nSolid academic record.\nStrong computer skills.\nKnowledge of other languages is desirable

In [116]:
df.to_csv("data_scientist_al_indeed_012021.csv")